# Custom Map File Creation notebook

This notebook will develop code to create the files that are necessary in the Beat Saber custom songs directory:

- .dat files (info.dat, 'level'.dat)
- cover.jpg file
- song.egg file

In [3]:
import numpy as np
import pandas as pd
import librosa
import json
import requests
import pickle
import matplotlib.pyplot as plt
from io import BytesIO, TextIOWrapper, StringIO
from zipfile import ZipFile
import os
import soundfile as sf
import audioread
from pydub import AudioSegment
from sklearn.multioutput import MultiOutputClassifier, ClassifierChain
from sklearn.ensemble import RandomForestClassifier

In [61]:
#{'_version': '2.0.0',
#  '_songName': 'Gamle Kjente_V03',
#  '_songSubName': 'Despacito',
#  '_songAuthorName': 'Vegard Bakkely',
#  '_levelAuthorName': 'Martin Ask Eriksen',
#  '_beatsPerMinute': 89,
#  '_songTimeOffset': 0,
#  '_shuffle': 0,
#  '_shufflePeriod': 0.5,
#  '_previewStartTime': 6,
#  '_previewDuration': 40,
#  '_songFilename': 'song.egg',
#  '_coverImageFilename': 'cover.jpg',
#  '_environmentName': 'DefaultEnvironment',
#  '_customData': {'_editor': 'beatmapper',
#   '_editorSettings': {'enabledFastWalls': False, 'modSettings': {}}},
#  '_difficultyBeatmapSets': [{'_beatmapCharacteristicName': 'Standard',
#    '_difficultyBeatmaps': [{'_difficulty': 'Easy',
#      '_difficultyRank': 1,
#      '_beatmapFilename': 'Easy.dat',
#      '_noteJumpMovementSpeed': 10,
#      '_noteJumpStartBeatOffset': 0,
#      '_customData': {'_editorOffset': 0, '_requirements': []}}]}]}

def write_info(song_name, bpm, difficulty):
    """This function creates the 'info.dat' file that needs to be included in the custom folder."""

    difficulty_rank = None
    jump_movement = None
    if difficulty.casefold() == 'easy'.casefold():
        difficulty_rank = 1
        jump_movement = 8
    elif difficulty.casefold() == 'normal'.casefold():
        difficulty_rank = 3
        jump_movement = 10
    elif difficulty.casefold() == 'hard'.casefold():
        difficulty_rank == 5
        jump_movement = 12
    elif difficulty.casefold() == 'expert'.casefold():
        difficulty_rank == 7
        jump_movement = 14
    elif difficulty.casefold() == 'expertPlus'.casefold():
        difficulty_rank == 9
        jump_movement = 16
            
    info = {'_version': '2.0.0',
            '_songName': f"{song_name}",
            '_songSubName': '',
            '_songAuthorName': '',
            '_levelAuthorName': 'BeatMapSynth',
            '_beatsPerMinute': round(bpm),
            '_songTimeOffset': 0,
            '_shuffle': 0,
            '_shufflePeriod': 0,
            '_previewStartTime': 10,
            '_previewDuration': 30,
            '_songFilename': 'song.egg',
            '_coverImageFilename': 'cover.jpg',
            '_environmentName': 'DefaultEnvironment',
            '_customData': {}, #I don't think anything is needed here
             '_difficultyBeatmapSets': [{'_beatmapCharacteristicName': 'Standard',
                                         '_difficultyBeatmaps': [{'_difficulty': f"{difficulty}",
                                                                  '_difficultyRank': difficulty_rank,
                                                                  '_beatmapFilename': f"{difficulty}.dat",
                                                                  '_noteJumpMovementSpeed': jump_movement, #not sure what this is, seems to vary with difficulty level, may want to 
                                                                  '_noteJumpStartBeatOffset': 0, #ditto
                                                                  '_customData': {}}]}]} #{'_editorOffset': 0, '_requirements': []}}]}]} - don't think this is needed
    with open('info.dat', 'w') as f:
        json.dump(info, f)


In [5]:
# {'_version': '2.0.0',
#  '_customData': {'_time': 220, '_BPMChanges': [], '_bookmarks': []},
#  '_events': [{'_time': 0, '_type': 1, '_value': 3}, LIST],
#  '_notes': [{'_time': 12.5,
#    '_lineIndex': 1,
#    '_lineLayer': 0,
#    '_type': 0,
#    '_cutDirection': 1}, LIST],
#  '_obstacles': [{'_time': 0,
#    '_lineIndex': 3,
#    '_type': 0,
#    '_duration': 9,
#    '_width': 1}, LIST]}

def write_level(difficulty, events_list, notes_list, obstacles_list):
    """This function creates the 'level.dat' file that contains all the data for that paticular difficulty level"""
    
    level = {'_version': '2.0.0',
             '_customData': {'_time': '', #not sure what time refers to 
                             '_BPMChanges': [], 
                             '_bookmarks': []},
             '_events': events_list,
             '_notes': notes_list,
             '_obstacles': obstacles_list}
    with open(f"{difficulty}.dat", 'w') as f:
        json.dump(level, f)

In [6]:
def beat_features(song_path):
    """This function takes in the song stored at 'song_path' and estimates the bpm and beat times."""
    #Load song and split into harmonic and percussive parts.
    y, sr = librosa.load(song_path)
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    #Isolate beats and beat times
    bpm, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)
    beat_times = librosa.frames_to_time(beat_frames, sr=sr)
    return bpm, beat_times

In [7]:
def music_file_converter(song_path):
    """This function makes sure the file type of the provided song will be converted to the music file type that 
    Beat Saber accepts"""
    if song_path.endswith('.mp3'):
        AudioSegment.from_mp3(song_path).export('song.egg', format='ogg')
    elif song_path.endswith('.wav'):
        AudioSegment.from_wav(song_path).export('song.egg', format='ogg')
    elif song_path.endswith('.flv'):
        AudioSegment.from_flv(song_path).export('song.egg', format='ogg')
    elif song_path.endswith('.raw'):
        AudioSegment.from_raw(song_path).export('song.egg', format='ogg')
    elif song_path.endswith('.ogg') or song_path.endswith('.egg'):
        pass
    else:
        print("Unsupported song file type. Choose a file of type .mp3, .wav, .flv, .raw, or .ogg.")


In [8]:
def random_notes_writer(beat_times, difficulty):
    """This function randomly places blocks at approximately each beat or every other beat depending on the difficulty."""
    notes_list = []
    line_index = [0, 1, 2, 3]
    line_layer = [0, 1, 2]
    types = [0, 1, 2, 3]
    directions = list(range(0, 10))
    beat_times = [float(x) for x in beat_times]
    
    if difficulty == 'Easy' or difficulty == 'Normal':
        for beat in beat_times:
            empty = np.random.choice([0,1])
            if empty == 1:
                note = {'_time': beat,
                        '_lineIndex': int(np.random.choice(line_index)),
                        '_lineLayer': int(np.random.choice(line_layer)),
                        '_type': int(np.random.choice(types)),
                        '_cutDirection': int(np.random.choice(directions))}
                notes_list.append(note)
            else:
                continue
    else:
        random_beats = np.random.choice(beat_times, np.random.choice(range(len(beat_times)))) #randomly choose beats to have more than one note placed
        randomly_duplicated_beat_times = np.concatenate([beat_times, random_beats])
        randomly_duplicated_beat_times.sort()
        randomly_duplicated_beat_times = [float(x) for x in randomly_duplicated_beat_times]
        for beat in randomly_duplicated_beat_times:
            note = {'_time': beat,
                    '_lineIndex': int(np.random.choice(line_index)),
                    '_lineLayer': int(np.random.choice(line_layer)),
                    '_type': int(np.random.choice(types)),
                    '_cutDirection': int(np.random.choice(directions))}
            notes_list.append(note)
    return notes_list

In [9]:
def random_notes_writer_v2(beat_times, difficulty):
    """This function randomly places blocks at approximately each beat or every other beat depending on the difficulty."""
    notes_list = []
    line_index = [0, 1, 2, 3]
    line_layer = [0, 1, 2]
    types = [0, 1, 2, 3]
    directions = list(range(0, 10))
    #beat_times = [float(x) for x in beat_times]
    beat_times = list(range(len(beat_times)))
    
    if difficulty == 'Easy' or difficulty == 'Normal':
        for beat in beat_times:
            empty = np.random.choice([0,1])
            if empty == 1:
                note = {'_time': beat,
                        '_lineIndex': int(np.random.choice(line_index)),
                        '_lineLayer': int(np.random.choice(line_layer)),
                        '_type': int(np.random.choice(types)),
                        '_cutDirection': int(np.random.choice(directions))}
                notes_list.append(note)
            else:
                continue
    else:
        random_beats = np.random.choice(beat_times, np.random.choice(range(len(beat_times)))) #randomly choose beats to have more than one note placed
        randomly_duplicated_beat_times = np.concatenate([beat_times, random_beats])
        randomly_duplicated_beat_times.sort()
        randomly_duplicated_beat_times = [float(x) for x in randomly_duplicated_beat_times]
        for beat in randomly_duplicated_beat_times:
            note = {'_time': beat,
                    '_lineIndex': int(np.random.choice(line_index)),
                    '_lineLayer': int(np.random.choice(line_layer)),
                    '_type': int(np.random.choice(types)),
                    '_cutDirection': int(np.random.choice(directions))}
            notes_list.append(note)
    return notes_list

In [10]:
def events_writer(beat_times):
    
    events_list = []
    return events_list

In [11]:
def obstacles_writer(beat_times, difficulty):
    
    obstacles_list = []
    return obstacles_list

In [12]:
def zip_folder_exporter(song_name, difficulty):
    "This function exports the zip folder containing the info.dat, difficulty.dat, cover.jpg, and song.egg files."
    files = ['info.dat', f"{difficulty}.dat", 'cover.jpg', 'song.egg']
    with ZipFile(f"{song_name}.zip", 'w') as custom:
        for file in files:
            custom.write(file)

In [13]:
def random_mapper(song_path, song_name, difficulty):
    """Function to output the automatically created completely random map (i.e., baseline model) for a provided song.
    Returns a zipped folder that can be unzipped and placed in the 'CustomMusic' folder in the Beat Saber game
    directory and played. CAUTION: This is completely random and is likely not enjoyable if even playable!"""
    #Load song and get beat features
    print("Loading Song...")
    bpm, beat_times = beat_features(song_path)
    print("Song loaded successfully!")
    #Write lists for note placement, event placement, and obstacle placement
    print("Random mapping...")
    #notes_list = random_notes_writer(beat_times, difficulty) 
    notes_list = random_notes_writer_v2(beat_times, difficulty) #fixes _time != beat time
    events_list = events_writer(beat_times)
    obstacles_list = obstacles_writer(beat_times, difficulty)
    print("Mapping done!")
    #Write and zip files
    print("Writing files to disk...")
    write_info(song_name, bpm, difficulty)
    write_level(difficulty, events_list, notes_list, obstacles_list)
    print("Converting music file...")
    music_file_converter(song_path)
    print("Zipping folder...")
    zip_folder_exporter(song_name, difficulty)
    print("Finished! Look for zipped folder in your current path, unzip the folder, and place in the 'CustomMusic' folder in the Beat Saber directory")

In [22]:
random_mapper('song.egg', 'example', 'expert')

Loading Song...


/opt/anaconda3/envs/BSmapsynth-env/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


Song loaded successfully!
Random mapping...
Mapping done!
Writing files to disk...
Converting music file...
Zipping folder...
Finished! Look for zipped folder in your current path, unzip the folder, and place in the 'CustomMusic' folder in the Beat Saber directory


In [13]:
random_mapper('./RandomExample/03 Toxic Bad Guy, Look What You Made Me Do (Billie Eilish vs. Taylor Swift vs. Britney Spears) - jimmylin.mp3', 'RandomExample', 'Normal')

Loading Song...


/opt/anaconda3/envs/BSmapsynth-env/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


Song loaded successfully!
Random mapping...
Mapping done!
Writing files to disk...
Converting music file...
Zipping folder...
Finished! Look for zipped folder in your current path, unzip the folder, and place in the 'CustomMusic' folder in the Beat Saber directory


In [41]:
def beat_number_and_chroma_16th_beat(song_path):
    """This function reads in a music file and returns a DataFrame of beat numbers (divided into 1/16th beats) and
    chroma features at each beat. Chroma features are calculated for 1/4 beats and propagated forward across the 
    16th beats."""
    #Load music file, estimate beat frames, and compute chromagram
    y, sr = librosa.load(song_path)
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    tempo, beat_frames = librosa.beat.beat_track(y=y_percussive,
                                             sr=sr,
                                             trim = False,
                                             units = 'frames')
    chromagram = librosa.feature.chroma_cqt(y=y_harmonic,
                                            sr=sr)
    #Make a framework for syncing chroma features to 1/4 beat
    extend = np.array([])
    ind = 0
    while ind < len(beat_frames)-1:
        extend = np.append(extend, np.arange(beat_frames[ind], beat_frames[ind+1], round((beat_frames[ind+1]-beat_frames[ind])/4))[1:4])
        ind += 1
    beat_frames_merged = np.concatenate((np.array([0]), beat_frames, extend))
    beat_frames_merged.sort()
    
    #Sync chroma features to 1/4 beats
    beat_chroma = librosa.util.sync(chromagram,
                                beat_frames_merged.astype(int),
                                aggregate=np.median)
    
    #Add beat count to beat frames
    t = np.arange(0, len(beat_frames), 1)
    beat_numbers = pd.concat([pd.Series(t, name = '_time'), pd.Series(beat_frames, name = 'frame_no')], axis = 1)
    beat_numbers['frame_no'] = beat_numbers['frame_no'].astype(int)
    
    #Merge chroma features with extended beat frames (1/4 beat)
    chromabeats = pd.concat([pd.Series(beat_frames_merged.astype(int), name = 'frame_no'), pd.DataFrame(beat_chroma.T)], axis = 1)
    z = pd.merge(beat_numbers, chromabeats, on = 'frame_no', how = 'outer', sort = True)
    z.interpolate(inplace = True)
    
    #Expand beat frequency to 1/16th beat & merge
    expand_time = pd.DataFrame(np.arange(0.00, z['_time'].max(), 0.0625), columns = ['_time'])
    expanded_chromabeats = pd.merge(z, expand_time, on='_time', how = 'outer', sort = True)
    
    #Forward fill to fill NaNs
    expanded_chromabeats.fillna(method='ffill', inplace = True)
    expanded_chromabeats.drop('frame_no', axis = 1, inplace = True)
    
    return tempo, expanded_chromabeats

In [53]:
def beat_number_and_chroma_half_beat(song_path):
    #Load music file, estimate beat frames, and compute chromagram
    y, sr = librosa.load(song_path)
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    tempo, beat_frames = librosa.beat.beat_track(y=y_percussive,
                                             sr=sr,
                                             trim = False,
                                             units = 'frames')
    chromagram = librosa.feature.chroma_cqt(y=y_harmonic,
                                            sr=sr)
    #Make a framework for syncing chroma features to 1/2 beat
    extend = np.array([])
    ind = 0
    while ind < len(beat_frames)-1:
        extend = np.append(extend, np.arange(beat_frames[ind], beat_frames[ind+1], round((beat_frames[ind+1]-beat_frames[ind])/2))[1:2])
        ind += 1
    beat_frames_merged = np.concatenate((np.array([0]), beat_frames, extend))
    beat_frames_merged.sort()
    
    #Sync chroma features to 1/2 beats
    beat_chroma = librosa.util.sync(chromagram,
                                beat_frames_merged.astype(int),
                                aggregate=np.median)
    
    #Add beat count to beat frames
    t = np.arange(0, len(beat_frames), 1)
    beat_numbers = pd.concat([pd.Series(t, name = '_time'), pd.Series(beat_frames, name = 'frame_no')], axis = 1)
    beat_numbers['frame_no'] = beat_numbers['frame_no'].astype(int)
    
    #Merge chroma features with extended beat frames (1/2 beat)
    chromabeats = pd.concat([pd.Series(beat_frames_merged.astype(int), name = 'frame_no'), pd.DataFrame(beat_chroma.T)], axis = 1)
    z = pd.merge(beat_numbers, chromabeats, on = 'frame_no', how = 'outer', sort = True)
    z.interpolate(inplace = True)
    z.drop('frame_no', axis = 1, inplace = True)
    z.drop(0, axis = 0, inplace = True)
    z.reset_index(drop=True, inplace = True)
    return tempo, z

In [59]:
def RFchain_notes_writer(music_features, difficulty):
    """This function takes in a music features DataFrame and difficulty level and returns a list of note placement
    dictionaries from the Random Forest chain classifier model's predictions."""
    #Load model
    with open(f"./models/chain_{difficulty}.pkl", 'rb') as m:
        model = pickle.load(m)
    #Make predictions using model
    preds = model[0].predict(music_features)
    #Make predictions into a dataframe and concatenate with beat times
    df_preds = pd.concat([music_features[['_time']], pd.DataFrame(preds, columns = model[1])], axis = 1, sort = True)
    #Write notes dictionaries
    notes_list = []
    for index, row in df_preds.iterrows():
        for x in list(filter(lambda y: y.startswith('notes_type'), df_preds.columns)):
            if row[x] != 999:
                num = x[-1]
                note = {'_time': row['_time'],
                        '_lineIndex': int(row[f"notes_lineIndex_{num}"]),
                        '_lineLayer': int(row[f"notes_lineLayer_{num}"]),
                        '_type': num,
                        '_cutDirection': int(row[f"notes_cutDirection_{num}"])}
                notes_list.append(note)
    return notes_list

    return df_preds

In [48]:
def RF_mapper(song_path, song_name, difficulty, model):
    #Load song and get beat features
    print("Loading Song...")
    bpm, music_features = beat_number_and_chroma_half_beat(song_path)
    print("Song loaded successfully!")
    #Write lists for note placement, event placement, and obstacle placement
    if model == 'chain':
        print("Mapping with Random Forest chain classifier model...")
        notes_list = RFchain_notes_writer(music_features, difficulty)
        events_list = events_writer(music_features['_time'])
        obstacles_list = obstacles_writer(music_features['_time'], difficulty)
        print("Mapping done!")
    else:
        print("Other model types are not implemented yet. Please use 'chain'.")
    #Write and zip files
    print("Writing files to disk...")
    write_info(song_name, bpm, difficulty)
    write_level(difficulty, events_list, notes_list, obstacles_list)
    print("Converting music file...")
    music_file_converter(song_path)
    print("Zipping folder...")
    zip_folder_exporter(song_name, difficulty)
    print("Finished! Look for zipped folder in your current path, unzip the folder, and place in the 'CustomMusic' folder in the Beat Saber directory")

In [60]:
RF_mapper('Captive_Portal_-_03_-_An_Example_For.mp3', 'An Example For - Captive Portal', 'expert', 'chain')

Loading Song...


/opt/anaconda3/envs/BSmapsynth-env/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


Song loaded successfully!
Mapping with Random Forest chain classifier model...
Mapping done!
Writing files to disk...
Converting music file...
Zipping folder...
Finished! Look for zipped folder in your current path, unzip the folder, and place in the 'CustomMusic' folder in the Beat Saber directory
